<a href="https://colab.research.google.com/github/youngeun1207/keras/blob/main/SHL/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def loadData(usr, day):
  dir_usr = ["user1", "user2", "user3"]
  dir_day = ["day1", "day2", "day3"]
  file_name = ["Hand_Motion.txt","Hand_GPS.txt","Hand_Location.txt", "Label.txt"]
  path = "/content/drive/MyDrive/SHLdatasets"

  motion_path = path + "/" + dir_usr[usr] + "/" + dir_day[day] + "/" + file_name[0]
  gps_path = path + "/" + dir_usr[usr] + "/" + dir_day[day] + "/" + file_name[1]
  location_path = path + "/" + dir_usr[usr] + "/" + dir_day[day] + "/" + file_name[2]
  label_path = path + "/" + dir_usr[usr] + "/" + dir_day[day] + "/" + file_name[3]

  print(motion_path)

  # 'Time', 'Acc_X', 'Acc_Y', 'Acc_Z', 'Gyr_X', 'Gyr_Y', 'Gyr_Z', 'Mag_X', 'Mag_Y', 'Mag_Z'
  full_motion = pd.read_csv(motion_path, sep=" ",header=None, usecols=list(range(10)))
  # 'Time', 'GPS_SNR' , 'GPS_Az, ', 'GPS_Elev', 
  full_gps = pd.read_csv(gps_path, sep=" ",header=None, usecols=[0,4,5,6])
  # 'Time, Accuracy, Latitude, Longitude, Altitude'
  full_location = pd.read_csv(location_path, sep=" ",header=None, usecols=[0,3,4,5,6])
  # Null=0, Still=1, Walking=2, Run=3, Bike=4, Car=5, Bus=6, Train=7, Subway=8
  full_label = pd.read_csv(label_path, sep=" ",header=None, usecols=[1])

  full_motion.columns = ['Time', 'Acc_X', 'Acc_Y', 'Acc_Z', 'Gyr_X', 'Gyr_Y', 'Gyr_Z', 'Mag_X', 'Mag_Y', 'Mag_Z']
  full_gps.columns = ['Time', 'GPS_SNR' , 'GPS_Az, ', 'GPS_Elev']
  full_location.columns = ['Time', 'Accuracy', 'Latitude', 'Longitude', 'Altitude']
  full_label.columns = ['Label']

  return full_motion, full_gps, full_location, full_label

In [ ]:
import pandas as pd
import numpy as np
from keras.utils.np_utils import to_categorical

def mergeData(full_motion, full_gps, full_location):
  full_motion.iloc[:, 0] = full_motion.iloc[:, 0].astype(int)
  full_gps.iloc[:, 0] = full_gps.iloc[:, 0] // 10 * 10
  full_location.iloc[:, 0] = full_location.iloc[:, 0] // 10 * 10

  full_data = pd.merge(full_motion, full_gps, on='Time', how='left')
  full_data = pd.merge(full_data, full_location, on='Time', how='left')
  
  full_data.fillna(method='ffill', limit=5, axis=0, inplace=True)
  full_data.fillna(method='bfill', limit=5, axis=0, inplace=True)
  full_data.drop_duplicates('Time', keep='first', inplace=True)

  return full_data

In [ ]:
def toNumpy(data, label):
  final_data = data.to_numpy()
  final_data = final_data.reshape(-1,1,17)

  label_np = label['Label'].T
  label_np = label_np.to_numpy()

  # one-hot encoding
  final_label = np.zeros(shape=(len(label_np), 9))
  
  label_one_hot = to_categorical(label_np)
  # label_one_hot = label_one_hot[0]
  for i in range(len(label_one_hot)):
    final_label[i] = np.pad(label_one_hot[i], (0,9 - len(label_one_hot[i])), constant_values = 0)

  return final_data, final_label

In [ ]:
def getData(usr, day):
  usr-=1
  day-=1
  full_motion, full_gps, full_location, label = loadData(usr, day)
  data = mergeData(full_motion, full_gps, full_location)
  np_data, np_label = toNumpy(data, label)
  return np_data, np_label

In [ ]:
# user 1,2 데이터로 훈련
for i in range(2):
  for j in range(3):
    if(i == 0 and j == 0):
      train_data, train_label = getData(1, 1)
      continue
    tmp_data, tmp_label = getData(i+1, j+1)
    train_data = np.concatenate([train_data, tmp_data])
    train_label = np.concatenate([train_label, tmp_label])
# user 3 데이터로 검증
for i in range(3):
  if i == 0:
    val_data, val_label = getData(3, i+1)
    continue
  tmp_data, tmp_label = getData(3, i+1)
  val_data = np.concatenate([val_data, tmp_data])
  val_label = np.concatenate([val_label, tmp_label])

print(train_data.shape)
print(train_label.shape)
print(val_data.shape)
print(val_label.shape)

NameError: ignored

In [ ]:
import numpy as np
train_data = np.load("/content/drive/MyDrive/SHLdatasets/train_data.npy")
val_data = np.load("/content/drive/MyDrive/SHLdatasets/val_data.npy")
train_label = np.load("/content/drive/MyDrive/SHLdatasets/train_label.npy")
val_label = np.load("/content/drive/MyDrive/SHLdatasets/val_label.npy")

In [ ]:
mean = np.nanmean(train_data, axis=0)
train_data = train_data - mean
val_data = val_data - mean
std = np.nanstd(train_data, axis=0)
train_data/=std
val_data/=std

In [ ]:
train_data[1750000//500][0]
#array([[-0.62667921, -1.7224187 ,  1.06512092,  0.03283842,  0.11032303,
        #  0.13222627,  0.01659528,  1.23409338, -0.63570765, -0.33971381,
        #         nan,         nan,         nan,         nan,         nan,
        #         nan,         nan]])

array([-0.62667413, -1.06847563,  1.21844673,  0.09503198, -0.09847013,
        0.31461889, -0.02015667,  1.16539573, -0.49253838, -0.35238838,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan])

In [ ]:
def divide_5sec(data, label):
  index = len(data)%500
  data = data[:-index].reshape((len(data)//500, 500, 17))
  label = label[:-index].reshape((len(label)//500, 500, 9))

  delete_index = []
  for i in range(len(label)//500):
    index = np.where(label[i][0]==1)
    for j in range(49):
      if np.where(label[i][j+1] != index):
        break
      delete_index.append(i)
  data = np.delete(data,(i), axis = 0)
  label = np.delete(label,(i), axis = 0)
  return data, label
val_data, val_label = divide_5sec(val_data, val_label)

In [ ]:
print(train_data.shape)
print(train_label.shape)
print(val_data.shape)
print(val_label.shape)

(42559, 500, 17)
(42559, 500, 9)
(17535, 500, 17)
(17535, 500, 9)


In [ ]:
np.save('/content/drive/MyDrive/SHLdatasets/train_data_5sec',train_data)
np.save('/content/drive/MyDrive/SHLdatasets/train_label_5sec',train_label)

np.save('/content/drive/MyDrive/SHLdatasets/val_data_5sec',val_data)
np.save('/content/drive/MyDrive/SHLdatasets/val_label_5sec',val_label)

---
# 여기서부터는 폐기 코드..

In [ ]:
import pandas as pd
import numpy as np
from keras.utils.np_utils import to_categorical

def getData(usr, day):
  dir_usr = ["user1", "user2", "user3"]
  dir_day = ["day1", "day2", "day3"]
  file_name = ["Hand_Motion.txt","Hand_GPS.txt","Label.txt"]
  path = "/content/drive/MyDrive/SHLdatasets"

  motion_path = path + "/" + dir_usr[usr-1] + "/" + dir_day[day-1] + "/" + file_name[0]
  gps_path = path + "/" + dir_usr[usr-1] + "/" + dir_day[day-1] + "/" + file_name[1]
  label_path = path + "/" + dir_usr[usr-1] + "/" + dir_day[day-1] + "/" + file_name[2]

  print(motion_path)

  # 'Time', 'Acc_X', 'Acc_Y', 'Acc_Z', 'Gyr_X', 'Gyr_Y', 'Gyr_Z', 'Mag_X', 'Mag_Y', 'Mag_Z'
  full_motion = pd.read_csv(motion_path, sep=" ",header=None, usecols=list(range(10)))
  # 'Time', 'GPS_SNR' , 'GPS_Az, ', 'GPS_Elev', 
  full_gps = pd.read_csv(gps_path, sep=" ",header=None, usecols=[0,4,5,6])
  # Null=0, Still=1, Walking=2, Run=3, Bike=4, Car=5, Bus=6, Train=7, Subway=8
  full_label = pd.read_csv(label_path, sep=" ",header=None, usecols=list(range(2)))

  # NaN 제거
  full_motion = full_motion.fillna(0)
  full_gps = full_gps.fillna(0)

  # 1sec 단위로 자르기
  time = -1
  idx2 = 0
  data_1sec = pd.DataFrame()
  label_1sec = pd.DataFrame()
  tmp1 = pd.DataFrame()
  tmp2 = pd.DataFrame()
  for idx1 in range(len(full_gps)):
    gps_time = full_gps.iloc[idx1, 0]//1000
    if gps_time - time >= 1:
      gap = gps_time - time
      time = gps_time
      idx2 += gap*100 if idx1 != 0 else 0
      if idx2 >= len(full_motion):
        break
      while time != full_motion.iloc[idx2, 0]//1000:
        idx2 += 1
      tmp1 = full_gps.iloc[idx1, 1:].T
      tmp2 = full_motion.iloc[idx2, 1:].T
      data_1sec = data_1sec.append(pd.concat([tmp1, tmp2],ignore_index=True),ignore_index=True)
      label_1sec = label_1sec.append(full_label.iloc[idx2, 1:],ignore_index=True)

  # 넘파이로 바꾸기
  final_data = data_1sec.to_numpy()
  final_data = final_data.reshape(-1,1,12)

  label_np = label_1sec.T
  label_np = label_np.to_numpy()

  # one-hot encoding
  final_label = np.zeros(shape=(len(label_np[0]), 9))
  
  label_one_hot = to_categorical(label_np)
  label_one_hot = label_one_hot[0]
  for i in range(len(label_one_hot)):
    final_label[i] = np.pad(label_one_hot[i], (0,9 - len(label_one_hot[i])), constant_values = 0)

  return final_data, final_label

In [ ]:
def divide_5sec_10ms(data_10ms, label_10ms):
  data_5sec = np.zeros((0, 500, 17))
  label_5sec = np.zeros((0, 9))
  tmp_data_5sec = np.zeros((0, 17))
  tmp_label_5sec = np.zeros((0, 9))
  cnt = 0
  for i in range(len(data_10ms)):
    if(i%10000==0):
        print(i)
    if cnt == 0:
      label = label_10ms[i]
      reshape_label = label.reshape((1, 9))
      reshape_data_10ms = data_10ms[i][0].reshape((1, 17))
      tmp_data_5sec = np.concatenate([tmp_data_5sec, reshape_data_10ms])
      cnt+=1
      continue
    if not all(label_10ms[i] == label): # 5초 이상 동일 이동수단 유지하지 않으면 버리기.
      tmp_data_5sec = np.zeros((0, 17)) # 0으로 초기화
      tmp_label_5sec = np.zeros((0, 9))
      cnt = 0
      continue
    reshape_data_10ms = data_10ms[i][0].reshape((1, 17))
    tmp_data_5sec = np.concatenate([tmp_data_5sec, reshape_data_10ms])
    if cnt == 499:
      reshape_data = tmp_data_5sec.reshape((1, 500, 17))
      data_5sec = np.concatenate([data_5sec, reshape_data])
      tmp_data_5sec = np.zeros((0, 17)) # 0으로 초기화
      label_5sec = np.concatenate([label_5sec, reshape_label])
      cnt = 0
      continue
    cnt+=1
  return data_5sec, label_5sec

train_data, train_label = divide_5sec_10ms(train_data, train_label)
val_data, val_label = divide_5sec_10ms(val_data, val_label)

8830000
8840000
8850000
8860000
8870000
8880000
8890000
8900000
8910000
8920000
8930000
8940000
8950000
8960000
8970000
8980000
8990000
9000000
9010000
9020000
9030000
9040000
9050000
9060000
9070000
9080000
9090000
9100000
9110000
9120000
9130000
9140000
9150000
9160000
9170000
9180000
9190000
9200000
9210000
9220000
9230000
9240000
9250000
9260000
9270000
9280000
9290000
9300000
9310000
9320000
9330000
9340000
9350000
9360000
9370000
9380000
9390000
9400000
9410000
9420000
9430000
9440000
9450000
9460000
9470000
9480000
9490000
9500000
9510000
9520000
9530000
9540000
9550000
9560000
9570000
9580000
9590000
9600000
9610000
9620000
9630000
9640000
9650000
9660000
9670000
9680000
9690000
9700000
9710000
9720000
9730000
9740000
9750000
9760000
9770000
9780000
9790000
9800000
9810000
9820000
9830000
9840000
9850000
9860000
9870000
9880000
9890000
9900000
9910000
9920000
9930000
9940000
9950000
9960000
9970000
9980000
9990000
10000000
10010000
10020000
10030000
10040000
10050000
10060000
1

KeyboardInterrupt: ignored